In [11]:
from backtesting import Backtest, Strategy
import yfinance as yf

import backtesting
backtesting.set_bokeh_output(notebook=False)

In [12]:
# https://stackoverflow.com/questions/40256338/calculating-average-true-range-atr-on-ohlc-data-with-python
def wwma(values, n):
    """
        J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df, n=14):
    data = df.copy()
    high = data.High
    low = data.Low
    close = data.Close
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    return atr

In [13]:
class FallUp(Strategy):
    def init(self):
        self.down_days = 0
        self.hold_days = 0

    def next(self):
        # close price of previous day
        pv = self.data.Close[-2]

        # close price of current day
        cv = self.data.Close[-1]

        # consecutively falling 3 times and reversal, buy-in
        if self.down_days >= 5 and cv > pv and self.data['atr'] > 4:
            self.buy()
            self.hold_days = 0
            self.down_days_after_position = 0

        # record falling times
        if cv < pv:
            self.down_days += 1
        else:
            self.down_days = 0

        if self.position:
            self.hold_days += 1

            # record falling times after position
            if cv < pv:
                self.down_days_after_position += 1

            if self.down_days > 1:
                self.position.close()

            # with position, falling once within 4 times, close the position
            if self.hold_days <= 4 and self.down_days_after_position > 0:
                self.position.close()

In [16]:
data = yf.download("^SPX", period="60d", interval="5m")
data['atr'] = atr(data)

bt = Backtest(
    data, 

    # Strategy
    FallUp, 

    # Commission
    commission=.002, 

    # market orders will be filled with respect to the current bar's closing price instead of the next bar's open.
    trade_on_close=True,

    # new order auto-closes the previous trade/position, making at most a single trade (long or short) in effect at each time
    exclusive_orders=True 
)

stats = bt.run()

[*********************100%%**********************]  1 of 1 completed


In [17]:
print(stats)

Start                     2023-11-27 09:30...
End                       2024-02-16 15:55...
Duration                     81 days 06:25:00
Exposure Time [%]                    0.899685
Equity Final [$]                  9737.898551
Equity Peak [$]                       10000.0
Return [%]                          -2.621014
Buy & Hold Return [%]               10.020946
Return (Ann.) [%]                  -11.079019
Volatility (Ann.) [%]                1.314512
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.621014
Avg. Drawdown [%]                   -2.621014
Max. Drawdown Duration       79 days 05:30:00
Avg. Drawdown Duration       79 days 05:30:00
# Trades                                   16
Win Rate [%]                              0.0
Best Trade [%]                      -0.113336
Worst Trade [%]                     -0.317105
Avg. Trade [%]                    

In [3]:
print(stats._trades)

NameError: name 'stats' is not defined

In [15]:
bt.plot()

/Users/kimfaicheang/workspace/snowball/venv/lib/python3.10/site-packages/backtesting/_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Row(id='3808', ...)